In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.utils import AnalysisException

In [2]:
spark = (SparkSession
         .builder
         .getOrCreate())

In [3]:
results = (
    spark.read.text("../../data/gutenberg_books/*.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z]*", 0).alias("word"))
    .where(F.col("word") != "")
    .groupby(F.col("word")).count()
    )

Por buena practica solo se debe ordenar antes de un show. Puede ser muy costoso estar ordenando datasets grandes

In [4]:
results.orderBy("count", ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| the|38895|
| and|23919|
|  of|21199|
|  to|20526|
|   a|14464|
|   i|13974|
|  in|12777|
|that| 9623|
|  it| 9099|
| was| 8920|
+----+-----+
only showing top 10 rows



In [5]:
try:
    results.coalesce(1).write.csv("./data/ch02-ch03_output/csv")
except AnalysisException as err:
    print(err)

try:
    results.coalesce(1).write.parquet("./data/ch02-ch03_output/parquet")
except AnalysisException as err:
    print(err)